In [ ]:
import cv2
import numpy as np
import copy
import os
import h5py
import copy
import pickle
import time
import glob

import multicam_toolbox as mct
import matplotlib.pyplot as plt
import yaml
import scipy.io
from scipy.optimize import linear_sum_assignment

from tqdm import tqdm

In [ ]:
def calc_dist(p1, p2):
    
    d = np.zeros([p1.shape[0],p2.shape[0]])
    d[:,:] = np.nan
    
    for i in range(p1.shape[0]):
        for j in range(p2.shape[0]):
            d[i,j] = np.sqrt(np.sum((p1[i,:]-p2[j,:])**2))

    d[np.isnan(d)] = 1e20
    
    return d

In [ ]:
def check_performance(data_name, result_dir, kf_range=[0,1500]):
    with open('./calib/config.yaml', 'r') as f:
        cfg = yaml.safe_load(f)
    ID = cfg['camera_id']

    annotation_file = './annotation/' + data_name + '.pickle'
    modelout_file = glob.glob(result_dir + '/' + data_name + '*/kp3d_fxdJointLen.pickle')
    modelout_file = modelout_file[0]
    fnum_file = glob.glob(result_dir + '/' + data_name + '*/' + str(ID[0]) + '/frame_num.npy')
    fnum_file = fnum_file[0]

    with open(annotation_file, 'rb') as f:
        anno = pickle.load(f)

    with open(modelout_file, 'rb') as f:
        mout = pickle.load(f)

    fnum = np.load(fnum_file)

    KF = np.array(anno['keyframe_fnum'])
    KF = KF[kf_range[0]:kf_range[1],0]

    X_pred = mout['kp3d']
    S_pred = mout['kp3d_score']

    cnt = 0
    R = []
    for i, kf in enumerate(KF):
        ii = i + kf_range[0]
        j = np.argwhere(fnum==kf).ravel()
        j = j[0]
        x_gt = anno['annotation'][ii]['p3d']
        skip_flag = anno['annotation'][ii]['skipped']

        if np.sum(skip_flag) > 0:
            #print(i, j, 'skipped')
            continue
        
        x_pred = np.mean(X_pred[:,j,5:7,:], axis=1)
        s_pred = np.nanmean(S_pred[:,j,:], axis=1)
        #s_pred = np.mean(S_pred[:,j,5:7], axis=1)

        I_pred = np.argwhere(s_pred>0).ravel()
        I_gt = np.argwhere(np.logical_not(np.isnan(x_gt[:,0]))).ravel()

        x_pred = x_pred[I_pred,:]
        x_gt = x_gt[I_gt,:]
        d = calc_dist(x_pred, x_gt)
        row_ind, col_ind = linear_sum_assignment(d)

        r = np.zeros(4)

        tp = 0
        for i_sub in range(col_ind.shape[0]):
            if I_gt[col_ind[i_sub]] == I_pred[row_ind[i_sub]]:
                if d[row_ind[i_sub], col_ind[i_sub]] < 400:
                    tp += 1 # true positive
        
        n_gt = x_gt.shape[0]
        n_pred = x_pred.shape[0]

        for i_sub in range(4):
            if i_sub in I_pred.tolist() and i_sub not in I_gt.tolist():
                x_exit = np.array([5000, 0, 800])
                d = np.sqrt(np.sum((x_pred[I_pred==i_sub,:]-x_exit)**2))
                if d < 1000:
                    n_pred -= 1

        R.append([tp, n_gt, n_pred])
        #print(i,j,r)


    R = np.array(R)

    RR = np.sum(R, axis=0)

    TP = RR[0]
    FN = RR[1] - TP
    FP = RR[2] - TP

    precision = TP/(TP+FP)
    recall = TP/(TP+FN)

    print(TP, FN, FP, precision, recall)

    return TP, FN, FP, precision, recall

In [ ]:

result_dir = './results3D'

data_names = ['gp1d_20211015_170000', 'gp1d_20211016_170000',
              'gp1d_20211017_170000','gp1d_20211019_170000',
              'mff1y_withlogger_20220617_170000', 'mff1y_withlogger_20220618_170000',
              'mff1y_withlogger_20220619_170000', 'mff1y_withlogger_20220620_170000',]
KF_range = [[5900, 6650], [3050, 3800], [0, 750], [3150,3900],
            [0, 750],[0, 750],[0, 750],[0, 750],]

result = np.zeros([len(data_names), 3])
for j, data_name in enumerate(data_names):
    print(data_name)
    TP, FN, FP = check_performance(data_name, result_dir, kf_range=KF_range[j])
    result[j,0] = TP
    result[j,1] = FN
    result[j,2] = FP

print('TOTAL:')
result_total = np.sum(result, axis=0)
TP = result_total[0]
FN = result_total[1]
FP = result_total[2]

precision = TP/(TP+FP)
recall = TP/(TP+FN)

print(TP, FN, FP, precision, recall)

In [2]:
import numpy
numpy.load('/mnt/nas_siddharth/code_final/results3D_sweep/gp1d_20211015_170000/MT0.65_PR0.30_TL0.05_TB36/gp1d_20211015_170000.2325-2705/kp3d.pickle', allow_pickle=True)

{'kp3d': array([[[[  394.50785625, -1754.99051012,   543.8613797 ],
          [  371.02714664, -1772.91955662,   557.34195322],
          [  376.04651053, -1777.57191314,   529.73852386],
          ...,
          [  416.29103562, -1646.34866579,   317.98250003],
          [  333.20677513, -1528.82886789,   226.45556585],
          [  390.66177069, -1531.04463924,   256.3173061 ]],
 
         [[  395.44319476, -1748.51819504,   538.35025088],
          [  377.40956755, -1768.74939215,   556.23705048],
          [  379.93168898, -1775.27682292,   528.67576567],
          ...,
          [  420.22956738, -1645.89648729,   321.78961086],
          [  341.61156895, -1528.46934157,   232.69989818],
          [  395.79008569, -1531.74604049,   257.50528783]],
 
         [[  394.65326636, -1742.93668848,   533.43733745],
          [  383.41550328, -1764.58699837,   554.88351295],
          [  383.69104767, -1772.89098352,   527.68796438],
          ...,
          [  424.12703104, -1645.44617313